In [1]:
import pandas as pd
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import joblib
import requests

def obter_dados_kucoin(par, intervalo='1hour'):
    url = f"https://api.kucoin.com/api/v1/market/candles?type={intervalo}&symbol={par}"
    response = requests.get(url)
    data = response.json()
    if data['code'] != '200000':
        return None
    registros = data['data']
    df = pd.DataFrame(registros, columns=['timestamp', 'open', 'close', 'high', 'low', 'volume', 'turnover'])
    df = df.iloc[::-1]
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    return df.dropna()

par = 'BTC-USDT'
df = obter_dados_kucoin(par)
df['RSI'] = RSIIndicator(df['close']).rsi()
bb = BollingerBands(df['close'])
df['Upper'] = bb.bollinger_hband()
df['Lower'] = bb.bollinger_lband()
df['pct_change'] = df['close'].pct_change()

# Definindo alvo: 0 = manter, 1 = compra, 2 = venda, com base na variação futura (próximo período)
df['target'] = df['pct_change'].shift(-1).apply(lambda x: 1 if x > 0.005 else (2 if x < -0.005 else 0))

df = df.dropna()

X = df[['close', 'RSI', 'Upper', 'Lower', 'pct_change']]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = DecisionTreeClassifier(max_depth=5, random_state=42)
clf.fit(X_train, y_train)

joblib.dump(clf, 'modelo_decision_tree.pkl')
print("Modelo treinado e salvo com sucesso.")


ModuleNotFoundError: No module named 'ta'